In [ ]:
# https://www.statology.org/cramers-v-in-python/
# https://stackoverflow.com/questions/20892799/using-pandas-calculate-cram%C3%A9rs-coefficient-matrix
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chi2_contingency.html
# https://stackoverflow.com/questions/51859894/how-to-plot-a-cramer-s-v-heatmap-for-categorical-features 
# https://stackoverflow.com/questions/67427325/cramer-v-correlation-in-python-but-instead-of-using-frequency-using-weights
# https://stackoverflow.com/questions/55609528/how-do-i-build-a-cramer-v-matrix-using-the-below-code
#
# https://stats.stackexchange.com/questions/7152/how-should-you-handle-cell-values-equal-to-zero-in-a-contingency-table
# https://www.kaggle.com/code/chrisbss1/cramer-s-v-correlation-matrix/notebook
# https://pbpython.com/pandas-crosstab.html

In [148]:
import pandas as pd
from tqdm import tqdm
from pysimilar import compare
import scipy.stats as stats
import numpy as np

pd.options.mode.chained_assignment = None  # default='warn'

#dir = "/content/"
dir = "/home/info/MyNotebooks/Datasets/SentencasTRT1/PDFs/Processed/Dataframe/"

df = pd.read_csv(dir+"cont_table_data_i_400.csv")
#df['entsFundamentacao'] = df['entsFundamentacao'].apply(eval)
df.head(12)

,Ents,Proc,Improc,Parcproc,Acordo
0,artigo 791 - A da CLT,81,11,203,0
1,artigo 467 da CLT,67,26,613,0
2,Lei 13. 467 / 2017,142,76,590,0
3,ADC nº 16,75,35,608,0
4,artigos 467 e 477 da CLT,124,30,429,0
5,"art. 71, 8 1º, da Lei nº 8. 666 / 93",43,22,332,0
6,Súmula 331 do TST,98,26,628,0
7,artigos 186 e 927 do Código Civil,45,16,361,0
8,artigo 883 da CLT,13,3,303,0
9,art. 2º da CLT,42,9,150,0


In [36]:
# Method 1:

#load necessary packages and functions


#create 2x2 table
data = np.array([[7,12], [9,8]])
data

array([[ 7, 12],
       [ 9,  8]])

In [64]:
data=df.iloc[:,1:].values    
print(data.shape)
print(data)

(400, 4)
[[ 81  11 203   0]
 [ 67  26 613   0]
 [142  76 590   0]
 ...
 [  2   3  26   0]
 [  5   0  25   0]
 [  9   0  22   0]]


In [65]:
# Deletar rows 0
df = df[(df[['Proc', 'Improc', 'Parcproc', 'Acordo']] != 0).any(axis=1)]
df.reset_index(drop=True, inplace=True) 
df

,Ents,Proc,Improc,Parcproc,Acordo
0,artigo 791 - A da CLT,81,11,203,0
1,artigo 467 da CLT,67,26,613,0
2,Lei 13. 467 / 2017,142,76,590,0
3,ADC nº 16,75,35,608,0
4,artigos 467 e 477 da CLT,124,30,429,0
...,...,...,...,...,...
391,IN 41 / 2018 do TST,6,3,23,0
392,"art. 1º, Lei 12. 506 / 11",2,2,27,0
393,artigo 2º da Medida Provisória 808 / 2017,2,3,26,0
394,0101863,5,0,25,0


In [73]:
df = df.drop('Acordo', axis=1)
df 

,Ents,Proc,Improc,Parcproc
0,artigo 791 - A da CLT,81,11,203
1,artigo 467 da CLT,67,26,613
2,Lei 13. 467 / 2017,142,76,590
3,ADC nº 16,75,35,608
4,artigos 467 e 477 da CLT,124,30,429
...,...,...,...,...
391,IN 41 / 2018 do TST,6,3,23
392,"art. 1º, Lei 12. 506 / 11",2,2,27
393,artigo 2º da Medida Provisória 808 / 2017,2,3,26
394,0101863,5,0,25


In [177]:
df['Proc'] = df['Proc'].astype(int)
df['Improc'] = df['Improc'].astype(int)
df['Parcproc'] = df['Parcproc'].astype(int)

In [178]:
df.dtypes

Ents        object
Proc         int64
Improc       int64
Parcproc     int64
Acordo       int64
dtype: object

In [179]:
df

,Ents,Proc,Improc,Parcproc,Acordo
0,artigo 791 - A da CLT,81,11,203,0
1,artigo 467 da CLT,67,26,613,0
2,Lei 13. 467 / 2017,142,76,590,0
3,ADC nº 16,75,35,608,0
4,artigos 467 e 477 da CLT,124,30,429,0
...,...,...,...,...,...
395,IN 41 / 2018 do TST,6,3,23,0
396,"art. 1º, Lei 12. 506 / 11",2,2,27,0
397,artigo 2º da Medida Provisória 808 / 2017,2,3,26,0
398,0101863,5,0,25,0


In [185]:
# creating crosstab data frame

tabs = pd.crosstab(df, columns=[['Ents'],['Proc','Improc','Parcproc']])
tabs.dtypes

ValueError: Shape of passed values is (5, 3), indices imply (400, 3)

In [176]:
# creating a statsmodels table object
import scipy.stats as stats
import statsmodels.api as sm
import numpy as np

table = sm.stats.Table(tabs)
table.table_orig

Proc,0,1,2,3,4,5,6,7,8,9,...,57,64,67,72,75,81,82,98,124,142
Ents,,,,,,,,,,,,,,,,,,,,,
$,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
$ 4º do art. 790 da CLT,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
$ 8º do artigo 477 da CLT,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
',1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
-,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
súmula nº 12 deste E. TRT,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
súmula nº 24 deste E. TRT,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
súmula nº 368,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [165]:
dfc = dfc.append(df['Ents'])
dfc = dfc.append(df['Proc'])
dfc.dtypes

dtype('O')

In [164]:
dfc.corr(method ='pearson')

TypeError: corr() missing 1 required positional argument: 'other'

In [145]:
df[df.columns[1:]].corr()['Proc']

AttributeError: 'Series' object has no attribute 'columns'

In [136]:

col1 = 'Ents'
col2 = 'Proc'
corr = dfc.corr(method ='pearson')
print("Correlation between ", col1, " and ", col2, "is: ", round(corr, 2))

KeyError: ('Ents', 'Proc')

In [112]:
g, p, dof, expctd = chi2_contingency(pd.crosstab(df['Ents'],df['Proc']))
g, p

(21383.999999999993, 0.3957152215978791)

In [103]:
from scipy.stats import chi2_contingency
chi2_contingency(df)

TypeError: '<' not supported between instances of 'str' and 'int'

In [77]:
g, p, dof, expctd = chi2_contingency(data, lambda_="log-likelihood")
g, p

ValueError: The internally computed table of expected frequencies has a zero element at (0, 3).

In [78]:
#Chi-squared test statistic, sample size, and minimum of rows and columns
X2 = stats.chi2_contingency(data, correction=False)[0]
X2

ValueError: The internally computed table of expected frequencies has a zero element at (0, 3).

In [113]:
n = np.sum(df)
minDim = min(df.shape)-1
n['Proc']
V = np.sqrt((g/n['Proc']) / minDim)

In [110]:

#calculate Cramer's V 
V = np.sqrt((X2/n['Proc']) / minDim)

In [114]:
#display Cramer's V
print(V)
#0.1617

1.2820032744362262


In [194]:
df1 = pd.crosstab(df['Ents'], df['Proc'])
df2 = pd.crosstab(df['Ents'], df['Improc'])
df3 = pd.crosstab(df['Ents'], df['Parcproc'])

pd.concat([df1, df2, df3], axis=1, keys=['Proc', 'Improc', 'Parcproc'])
g = df.groupby("Ents")
g["Proc"].value_counts()
g["Improc"].value_counts()
g["Parcproc"].value_counts()
pd.concat([g["comp"].value_counts().unstack(1), g["precedingWord"].value_counts().unstack(1)]).fillna(0)

Ents                       Proc
$                          0       1
$ 4º do art. 790 da CLT    4       1
$ 8º do artigo 477 da CLT  8       1
'                          0       1
-                          0       1
                                  ..
súmula nº 12 deste E. TRT  17      1
súmula nº 24 deste E. TRT  18      1
súmula nº 368              4       1
súmulas 200 e 211 do TST   2       1
taxa Selic                 6       1
Name: Proc, Length: 400, dtype: int64

In [ ]:
pd.concat([g["comp"].value_counts().unstack(1), g["precedingWord"].value_counts().unstack(1)]).fillna(0)

In [215]:
dfc = pd.crosstab(df['Ents'], df['Proc'])
stat,p,dof,expec = chi2_contingency(dfc)
n = df.sum()['Proc']
r, k = df.shape
phi2 = stat/n
phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))    
rcorr = r - ((r-1)**2)/(n-1)
kcorr = k - ((k-1)**2)/(n-1)
V = np.sqrt(phi2corr / min( (kcorr-1), (rcorr-1)))
V
# Proc = 1.074
# Parcproc = 0.66
# Improc = 1.2540

1.0743112741142047

In [ ]:
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum()['Proc']
    phi2 = chi2/n
    r,k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))    
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min( (kcorr-1), (rcorr-1)))

In [91]:
from scipy.stats import chi2_contingency
stat,p,dof,expec = chi2_contingency(df['Ents'],df['Proc','Improc','Parcproc'])
print(stat,p,dof,expec)

KeyError: ('Proc', 'Improc', 'Parcproc')

In [128]:
confusion_matrix = df['Ents','Proc']
confusion_matrix

KeyError: ('Ents', 'Proc')

In [120]:
# Method 2:

import scipy.stats as ss

def cramers_corrected_stat(confusion_matrix):
    """ calculate Cramers V statistic for categorial-categorial association.
        uses correction from Bergsma and Wicher, 
        Journal of the Korean Statistical Society 42 (2013): 323-328
    """
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum()['Proc']
    phi2 = chi2/n
    r,k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))    
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min( (kcorr-1), (rcorr-1)))

In [121]:
cramers_corrected_stat(confusion_matrix)

KeyError: 'Proc'

In [50]:
data

array([[ 7, 12],
       [ 9,  8]])